In [3]:
import pandas as pd
import pyarrow as pa
import pyarrow.parquet as pq

In [7]:
df =  pd.read_csv('LABEVENTS.csv')
df.tail(10)

,row_id,subject_id,hadm_id,itemid,charttime,value,valuenum,valueuom,flag
76064,20452674,44228,103379.0,51301,2170-12-23 04:36:00,16.8,16.80,K/uL,abnormal
76065,20452675,44228,103379.0,51221,2170-12-24 04:09:00,24.2,24.20,%,abnormal
76066,20452676,44228,103379.0,51222,2170-12-24 04:09:00,8.3,8.30,g/dL,abnormal
76067,20452677,44228,103379.0,51248,2170-12-24 04:09:00,30.2,30.20,pg,NaN
76068,20452678,44228,103379.0,51249,2170-12-24 04:09:00,34.4,34.40,%,NaN
76069,20452679,44228,103379.0,51250,2170-12-24 04:09:00,88,88.00,fL,NaN
76070,20452680,44228,103379.0,51265,2170-12-24 04:09:00,595,595.00,K/uL,abnormal
76071,20452681,44228,103379.0,51277,2170-12-24 04:09:00,14.5,14.50,%,NaN
76072,20452682,44228,103379.0,51279,2170-12-24 04:09:00,2.76,2.76,m/uL,abnormal
76073,20452683,44228,103379.0,51301,2170-12-24 04:09:00,16.9,16.90,K/uL,abnormal


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 76074 entries, 0 to 76073
Data columns (total 9 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   row_id      76074 non-null  int64  
 1   subject_id  76074 non-null  int64  
 2   hadm_id     61812 non-null  float64
 3   itemid      76074 non-null  int64  
 4   charttime   76074 non-null  object 
 5   value       76069 non-null  object 
 6   valuenum    67030 non-null  float64
 7   valueuom    66669 non-null  object 
 8   flag        29737 non-null  object 
dtypes: float64(2), int64(3), object(4)
memory usage: 5.2+ MB


In [9]:
df.isna().sum()

row_id            0
subject_id        0
hadm_id       14262
itemid            0
charttime         0
value             5
valuenum       9044
valueuom       9405
flag          46337
dtype: int64

In [10]:
df['charttime'] = pd.to_datetime(df['charttime'],errors='coerce').dt.strftime('%Y-%m-%d %H:%M:%S')
df['hadm_id'].fillna(-1, inplace=True)
df['hadm_id'] =df['hadm_id'].astype('int64')


C:\Users\LEGION\AppData\Local\Temp\ipykernel_28572\797406718.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['hadm_id'].fillna(-1, inplace=True)


In [11]:
df['charttime'].fillna('9999-12-31 00:00:00', inplace=True)
df['flag'].fillna('Normal', inplace=True)
df['valueuom'].fillna('Unknown', inplace=True)

df['value'].fillna('Unknown', inplace=True)
df['valuenum'].fillna(-1, inplace=True)

C:\Users\LEGION\AppData\Local\Temp\ipykernel_28572\2461630767.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['charttime'].fillna('9999-12-31 00:00:00', inplace=True)
C:\Users\LEGION\AppData\Local\Temp\ipykernel_28572\2461630767.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a c

In [12]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 76074 entries, 0 to 76073
Data columns (total 9 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   row_id      76074 non-null  int64  
 1   subject_id  76074 non-null  int64  
 2   hadm_id     76074 non-null  int64  
 3   itemid      76074 non-null  int64  
 4   charttime   76074 non-null  object 
 5   value       76074 non-null  object 
 6   valuenum    76074 non-null  float64
 7   valueuom    76074 non-null  object 
 8   flag        76074 non-null  object 
dtypes: float64(1), int64(4), object(4)
memory usage: 5.2+ MB


In [13]:
df.isna().sum()

row_id        0
subject_id    0
hadm_id       0
itemid        0
charttime     0
value         0
valuenum      0
valueuom      0
flag          0
dtype: int64

In [14]:
table = pa.Table.from_pandas(df)
pq.write_table(table, 'LABEVENTS.parquet')

In [ ]:
parquet_schema = pq.read_table('LABEVENTS.parquet').schema
print([(field.name, field.type) for field in parquet_schema])

[('row_id', DataType(int64)), ('subject_id', DataType(int64)), ('hadm_id', DataType(int64)), ('itemid', DataType(int64)), ('charttime', DataType(string)), ('value', DataType(string)), ('valuenum', DataType(double)), ('valueuom', DataType(string)), ('flag', DataType(string))]
